# Airbnb Price Analysis - Data Wrangling
### Springboard Data Science - Third Capstone - Travis Martin

In [733]:
# Importing necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import nltk
nltk.download('punkt')
from datetime import timedelta
import os
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
nltk.download('stopwords')
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [734]:
# Loading training dataset to Pandas Dataframe
df = pd.read_csv('D://Springboard/ThirdCapstone/RawData/AirbnbTrainSet.csv')

C:\Users\Admin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,21,22,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [735]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99569 entries, 0 to 99568
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      74111 non-null  float64
 1   log_price               74111 non-null  float64
 2   property_type           74111 non-null  object 
 3   room_type               74111 non-null  object 
 4   amenities               74111 non-null  object 
 5   accommodates            74111 non-null  float64
 6   bathrooms               73926 non-null  float64
 7   bed_type                74111 non-null  object 
 8   cancellation_policy     74111 non-null  object 
 9   cleaning_fee            74111 non-null  object 
 10  city                    74111 non-null  object 
 11  description             74111 non-null  object 
 12  first_review            58247 non-null  object 
 13  host_has_profile_pic    73923 non-null  object 
 14  host_identity_verified  73923 non-null

In [736]:
df.head(10)

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257.0,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3.0,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2.0,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928.0,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7.0,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6.0,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400.0,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5.0,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10.0,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779.0,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4.0,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0.0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117,2.0,2.0
4,3808709.0,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2.0,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4.0,40.0,NaN,20009,0.0,1.0
5,12422935.0,4.442651,Apartment,Private room,"{TV,""Wireless Internet"",Heating,""Smoke detecto...",2.0,1.0,Real Bed,strict,True,...,37.753164,-122.429526,Comfort Suite San Francisco,Noe Valley,3.0,100.0,https://a0.muscache.com/im/pictures/82509143-4...,94131,1.0,1.0
6,11825529.0,4.418841,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",3.0,1.0,Real Bed,moderate,True,...,33.980454,-118.462821,Beach Town Studio and Parking!!!11h,NaN,15.0,97.0,https://a0.muscache.com/im/pictures/4c920c60-4...,90292,1.0,1.0
7,13971273.0,4.787492,Condominium,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Wheelchair...",2.0,1.0,Real Bed,moderate,True,...,34.046737,-118.260439,"Near LA Live, Staple's. Starbucks inside. OWN ...",Downtown,9.0,93.0,https://a0.muscache.com/im/pictures/61bd05d5-c...,90015,1.0,1.0
8,180792.0,4.787492,House,Private room,"{TV,""Cable TV"",""Wireless Internet"",""Pets live ...",2.0,1.0,Real Bed,moderate,True,...,37.781128,-122.501095,Cozy Garden Studio - Private Entry,Richmond District,159.0,99.0,https://a0.muscache.com/im/pictures/0ed6c128-7...,94121,1.0,1.0
9,5385260.0,3.583519,House,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",2.0,1.0,Real Bed,moderate,True,...,33.992563,-117.895997,No.7 Queen Size Cozy Room 舒适大床房,NaN,2.0,90.0,https://a0.muscache.com/im/pictures/8d2f08ce-b...,91748,1.0,1.0


As loaded, this dataset consists of 29 columns and just over 74K rows.
The columns are a combination of objects (text strings) and floats (numerical data), and approximately half contain some proportion of null values, which need to be addressed.

Our most likely target column, log_price is the natural log of the listing price for that property.

One of the first things we notice by looking at the first 10 rows of the data is that each property has a unique id, which will help with our analysis. We also notice that our amenities column (which is perhaps our most important) holds a string containing the full list of amenities wrapped in curly brackets. Also of note is that amenities longer than a single word are wrapped in quotes to protect their meaning.

But before we start unpacking our amenities, our business use case has some constraints which we need to reflect in the data. The first of these is city, as we're restricting our analysis to only properties in New York City.

In [737]:
df['city'].unique()

array(['NYC', 'SF', 'DC', 'LA', 'Chicago', 'Boston', nan], dtype=object)

By inspecting the unique values of the city column, we see that the dataset also includes properties from 5 other major cities. We'll filter for only NYC properties and create a new dataframe.

In [738]:
df_NYC = df[df.city == 'NYC']
df_NYC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32349 entries, 0 to 74109
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      32349 non-null  float64
 1   log_price               32349 non-null  float64
 2   property_type           32349 non-null  object 
 3   room_type               32349 non-null  object 
 4   amenities               32349 non-null  object 
 5   accommodates            32349 non-null  float64
 6   bathrooms               32265 non-null  float64
 7   bed_type                32349 non-null  object 
 8   cancellation_policy     32349 non-null  object 
 9   cleaning_fee            32349 non-null  object 
 10  city                    32349 non-null  object 
 11  description             32349 non-null  object 
 12  first_review            25491 non-null  object 
 13  host_has_profile_pic    32173 non-null  object 
 14  host_identity_verified  32173 non-null

This filtering has left us with just under half of the original records.

Another constraint needed for our analysis is the type of property, as our comparison properties are full apartments and houses. We therefore need to remove listings that are for a single room only. We'll filter for 'Entire home/apt', and update our dataframe.

In [739]:
df_NYC.room_type.unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [740]:
df_NYC = df_NYC[df_NYC.room_type == 'Entire home/apt']

In [741]:
df_NYC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16243 entries, 0 to 74109
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      16243 non-null  float64
 1   log_price               16243 non-null  float64
 2   property_type           16243 non-null  object 
 3   room_type               16243 non-null  object 
 4   amenities               16243 non-null  object 
 5   accommodates            16243 non-null  float64
 6   bathrooms               16227 non-null  float64
 7   bed_type                16243 non-null  object 
 8   cancellation_policy     16243 non-null  object 
 9   cleaning_fee            16243 non-null  object 
 10  city                    16243 non-null  object 
 11  description             16243 non-null  object 
 12  first_review            13222 non-null  object 
 13  host_has_profile_pic    16163 non-null  object 
 14  host_identity_verified  16163 non-null

Looking at dataframe info again, we're now down to approximately 16K rows for our analysis. With our data constrained, we're ready to begin extracting our amenities.

In [742]:
df_NYC['amenities'] = df_NYC['amenities'].str.strip('{}')
# df_NYC.to_csv('D:/Springboard/ThirdCapstone/dataframe.csv')


Our first priority is to create a list of all of the possible amenities, as well as their relative frequency in the data.

In [743]:
# Creating a list of unique amenity values.

list_all = df_NYC['amenities'].str.replace('["{}]', "", regex=True).str.split(",").dropna().to_numpy()
list_unique = np.unique(sum(list_all, []))

# unique, counts = np.unique(list_unique, return_counts=True)

# def convert(list):
#    return tuple(list)

#convert(unique)
#convert(counts)

#unique

# dict(zip(unique, counts))

print(list_unique)
print(np.size(list_unique))

['' ' smooth pathway to front door' '24-hour check-in'
 'Accessible-height bed' 'Accessible-height toilet' 'Air conditioning'
 'Air purifier' 'BBQ grill' 'Baby bath' 'Baby monitor'
 'Babysitter recommendations' 'Bathtub' 'Bathtub with shower chair'
 'Beach essentials' 'Beachfront' 'Bed linens' 'Breakfast'
 'Buzzer/wireless intercom' 'Cable TV' 'Carbon monoxide detector' 'Cat(s)'
 'Changing table' 'Children’s books and toys' 'Children’s dinnerware'
 'Cleaning before checkout' 'Coffee maker' 'Cooking basics' 'Crib'
 'Disabled parking spot' 'Dishes and silverware' 'Dishwasher' 'Dog(s)'
 'Doorman' 'Dryer' 'EV charger' 'Elevator' 'Essentials'
 'Ethernet connection' 'Extra pillows and blankets' 'Family/kid friendly'
 'Fire extinguisher' 'Fireplace guards' 'Firm mattress' 'First aid kit'
 'Fixed grab bars for shower & toilet' 'Flat' 'Free parking on premises'
 'Free parking on street' 'Game console' 'Garden or backyard'
 'Ground floor access' 'Gym' 'Hair dryer' 'Handheld shower head' 'Hangers

In [744]:
# Using dummy variables (0/1) to calculate # of occurrences of each of the unique amenities.

df_counts = df_NYC.amenities.str.get_dummies(',').replace(0,np.nan).stack().sum(level=1)
df_counts.to_csv('D:/Springboard/ThirdCapstone/amenity_counts.csv')
print(df_counts)

"Air conditioning"                              14688.0
"Family/kid friendly"                            9386.0
"Hair dryer"                                     9554.0
"Wireless Internet"                             15808.0
"translation missing: en.hosting_amenity_50"     4373.0
                                                 ...   
"Private bathroom"                                  3.0
"Free parking on street"                            7.0
"Washer / Dryer"                                    5.0
"Bathtub with shower chair"                        15.0
"Roll-in shower with chair"                         1.0
Length: 116, dtype: float64


There are a total of 116 unique amenities (and one 'blank') present in the dataset. To make our analysis more targeted and reduce dimensionality, we will assign each amenity to one of three basic categories. We will leverage the first group, but not the other two.

**Targets** - Amenities which potentially drive a price increase (or decrease) and are also prevalent in sufficient numbers in the source data.
   - Wireless Internet / Internet
   - Air conditioning
   - Laptop friendly workspace
   - Family/kid friendly
   - Washer / Dryer / 'Washer / Dryer'
   - Elevator
   - 24-hour check-in
   - Lock on bedroom door
   - Self Check-In
   - Pets allowed
   - Doorman
   - Private entrance
   - Stove
   - Oven
   - Gym
   - Lockbox
   - Free parking on premises / Free parking on street
   - Breakfast
   - Wheelchair accessible
   - Bathtub
   - Pets live on this property
   - Suitable for events
   - Indoor fireplace
   - Dishwasher
   - Hot tub
   - Long term stays allowed
   - Room-darkening shades
   - Dog(s) / Cat(s) / Other pet(s)
   - Luggage dropoff allowed
   - Smoking allowed
   - Keypad
   - Patio or balcony
   - Garden or backyard
   - Private living room
   - Pool
   - Smart lock
   - BBQ grill
   - Crib / High chair / Pack â€™n Play/travel crib


**"Givens"** - Amenities which are either considered a given in a short-term rental (e.g. Smoke Detector, Shampoo), or are small enough "nice to have" upgrades that they will likely not measurably impact price (e.g. First aid kit, Outlet covers)
   - Kitchen
   - Heating
   - Essentials
   - Smoke detector
   - TV
   - Shampoo
   - Carbon monoxide detector
   - Hangers
   - Hair dryer
   - Iron
   - Buzzer/wireless intercom
   - Cable TV
   - Fire extinguisher
   - First aid kit
   - translation missing: en.hosting_amenity_50
   - translation missing: en.hosting_amenity_49
   - Safety card
   - Refrigerator
   - Dishes and silverware
   - Cooking basics
   - Hot water
   - Bed linens
   - Microwave
   - Coffee maker
   - Extra pillows and blankets
   - Host greets you
   - Childrenâ€™s books and toys
   - Step-free access
   - Window guards
   - Well-lit path to entrance
   - Ethernet connection
   - Babysitter recommendations
   - Single level home
   - Wide doorway
   - Childrenâ€™s dinnerware
   - Wide hallway clearance
   - Wide clearance to bed
   - Smooth pathway to front door
   - Flat
   - Outlet covers
   - Accessible-height bed
   - Wide entryway
   - Accessible-height toilet
   - Baby bath
   - Other
   - Pocket wifi
   - Changing table
   - Stair gates
   - Hot water kettle
   - Firm mattress
   - Wide clearance to shower & toilet
   - Handheld shower head
   - Fireplace guards
   - Baby monitor
   - Table corner guards
   - Fixed grab bars for shower & toilet
   - Bathtub with shower chair

**Exclusions** - Amenities which could potentially drive a measurable price delta, but are extremely low in number (e.g. EV Charger) or inappropriate for the NYC market (e.g. Ski in/Ski out).
   - Game console
   - Cleaning before checkout
   - Waterfront
   - Beach essentials
   - EV charger
   - Ground floor access
   - Disabled parking spot
   - Beachfront
   - Air purifier
   - Lake access
   - Private bathroom
   - Ski in/Ski out
   - Roll-in shower with chair


We also see two data-gathering errors in these results: 'translation missing: en.hosting_amenity_49 and 50. We will need do some digging to see if we can find these amenity titles online, otherwise we'll likely need to exclude them.

Now all that remains is to use string operations to create new dummy columns for our Target amenities; One for each amenity, set to either 1 if that property has it, or 0 if it does not. In all, we'll be adding 38 new binary columns.

In [745]:
df_NYC['AC'] = np.where(df_NYC['amenities'].str.contains("Air conditioning"), 1, 0)
df_NYC['WiFi'] = np.where(df_NYC['amenities'].str.contains("Internet"), 1, 0)
df_NYC['Office'] = np.where(df_NYC['amenities'].str.contains("Laptop friendly"), 1, 0)
df_NYC['Kids'] = np.where(df_NYC['amenities'].str.contains("kid friendly"), 1, 0)
df_NYC['Washer'] = np.where(df_NYC['amenities'].str.contains("Washer"), 1, 0)
df_NYC['Dryer'] = np.where(df_NYC['amenities'].str.contains("Dryer"), 1, 0)
df_NYC['Elevator'] = np.where(df_NYC['amenities'].str.contains("Elevator"), 1, 0)
df_NYC['24hr'] = np.where(df_NYC['amenities'].str.contains("24-hour"), 1, 0)
df_NYC['BR_Lock'] = np.where(df_NYC['amenities'].str.contains("Lock on bedroom"), 1, 0)
df_NYC['Self_CheckIn'] = np.where(df_NYC['amenities'].str.contains("Self Check-In"), 1, 0)
df_NYC['Pet_Friendly'] = np.where(df_NYC['amenities'].str.contains("Pets allowed"), 1, 0)
df_NYC['Doorman'] = np.where(df_NYC['amenities'].str.contains("Doorman"), 1, 0)
df_NYC['Private_Ent'] = np.where(df_NYC['amenities'].str.contains("Private entrance"), 1, 0)
df_NYC['Stove'] = np.where(df_NYC['amenities'].str.contains("Stove"), 1, 0)
df_NYC['Oven'] = np.where(df_NYC['amenities'].str.contains("Oven"), 1, 0)
df_NYC['Gym'] = np.where(df_NYC['amenities'].str.contains("Gym"), 1, 0)
df_NYC['Lockbox'] = np.where(df_NYC['amenities'].str.contains("Lockbox"), 1, 0)
df_NYC['Free_parking'] = np.where(df_NYC['amenities'].str.contains("Free parking"), 1, 0)
df_NYC['Breakfast'] = np.where(df_NYC['amenities'].str.contains("Breakfast"), 1, 0)
df_NYC['Wheelchair'] = np.where(df_NYC['amenities'].str.contains("Wheelchair"), 1, 0)
df_NYC['Pets_on_prop'] = np.where(df_NYC['amenities'].str.contains("Pets live"), 1, 0)
df_NYC['Events'] = np.where(df_NYC['amenities'].str.contains("events"), 1, 0)
df_NYC['Fireplace'] = np.where(df_NYC['amenities'].str.contains("fireplace"), 1, 0)
df_NYC['Dishwasher'] = np.where(df_NYC['amenities'].str.contains("Dishwasher"), 1, 0)
df_NYC['Hot_tub'] = np.where(df_NYC['amenities'].str.contains("Hot tub"), 1, 0)
df_NYC['Long_term'] = np.where(df_NYC['amenities'].str.contains("Long term stays"), 1, 0)
df_NYC['Darkening'] = np.where(df_NYC['amenities'].str.contains("Room-darkening"), 1, 0)
df_NYC['Pets'] = np.where(df_NYC['amenities'].str.contains("(s)"), 1, 0)
df_NYC['Luggage'] = np.where(df_NYC['amenities'].str.contains("Luggage dropoff"), 1, 0)
df_NYC['Smoking'] = np.where(df_NYC['amenities'].str.contains("Smoking allowed"), 1, 0)
df_NYC['Keypad'] = np.where(df_NYC['amenities'].str.contains("Keypad"), 1, 0)
df_NYC['Patio_Balcony'] = np.where(df_NYC['amenities'].str.contains("Patio or balcony"), 1, 0)
df_NYC['Garden_backyard'] = np.where(df_NYC['amenities'].str.contains("Garden or backyard"), 1, 0)
df_NYC['Private_LR'] = np.where(df_NYC['amenities'].str.contains("Private living room"), 1, 0)
df_NYC['Pool'] = np.where(df_NYC['amenities'].str.contains("Pool"), 1, 0)
df_NYC['Smart lock'] = np.where(df_NYC['amenities'].str.contains("Smart lock"), 1, 0)
df_NYC['BBQ grill'] = np.where(df_NYC['amenities'].str.contains("BBQ grill"), 1, 0)
df_NYC['Crib'] = np.where(df_NYC['amenities'].str.contains("Crib"), 1, 0)

df_NYC.head()

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,Luggage,Smoking,Keypad,Patio_Balcony,Garden_backyard,Private_LR,Pool,Smart lock,BBQ grill,Crib
0,6901257.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",3.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
1,6304928.0,5.129899,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",7.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
2,7919400.0,4.976734,Apartment,Entire home/apt,"TV,""Cable TV"",""Wireless Internet"",""Air conditi...",5.0,1.0,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,0,0
18,17589436.0,4.882802,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
19,18224863.0,4.595120,House,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",8.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0


In [746]:
df_NYC.to_csv('D:\Springboard\ThirdCapstone\Amenities.csv')

Next we turn our attention to null values. We'll also be checking for data fidelity in these steps as well (ex: are all zipcode entries formatted identically?) 

In [747]:
df_NYC.columns[df_NYC.isnull().any()].tolist()

['bathrooms',
 'first_review',
 'host_has_profile_pic',
 'host_identity_verified',
 'host_response_rate',
 'host_since',
 'last_review',
 'neighbourhood',
 'review_scores_rating',
 'thumbnail_url',
 'zipcode',
 'beds']

We'll attack these columns in order, starting with bathrooms.

In [748]:
df_NYC['bathrooms'].value_counts()

1.0    14027
2.0     1296
1.5      468
2.5      213
3.0      107
4.0       40
3.5       35
4.5       10
5.0       10
0.5        8
0.0        5
5.5        4
6.0        3
8.0        1
Name: bathrooms, dtype: int64

In [749]:
df_NYC.loc[df_NYC['bathrooms'].isnull()]

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,Luggage,Smoking,Keypad,Patio_Balcony,Garden_backyard,Private_LR,Pool,Smart lock,BBQ grill,Crib
1351,13928644.0,5.164786,Apartment,Entire home/apt,,2.0,NaN,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
5462,7700522.0,5.703782,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,NaN,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
8761,13602504.0,5.247024,Apartment,Entire home/apt,"Internet,""Wireless Internet"",""Air conditioning...",4.0,NaN,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
10957,15985033.0,5.273000,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,NaN,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
14714,14672035.0,4.779123,Apartment,Entire home/apt,"TV,""Wireless Internet"",Kitchen,Doorman,Heating",2.0,NaN,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,0,0
24208,17126504.0,4.787492,Apartment,Entire home/apt,,2.0,NaN,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0,0
31904,8293942.0,5.075174,Apartment,Entire home/apt,,3.0,NaN,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0,0
35433,15588469.0,5.164786,Apartment,Entire home/apt,"Internet,""Wireless Internet"",""Air conditioning...",3.0,NaN,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,0,0
40052,15177667.0,5.164786,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",4.0,NaN,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
49593,5569521.0,4.634729,Apartment,Entire home/apt,,2.0,NaN,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0,0


Based on value counts, we see that # of bathrooms ranges from 0 to 8, with the vast majority at 1. 5 properties are listed as having 0 bathrooms, and 16 more are null.   

Digging into the descriptions of these listings, we see that of the 21 coded as 'no bathroom', only 2 actually confirm that there is no bathroom on the property (one is a houseboat - id=21164481, and the other is a small garage apartment id=3914363). Neither of these types of property is particularly representative of our comparison properties, so we'll remove these two rows and then convert the remaining 19 listings to 1 bath, as that's the most likely value.

There are also 8 properties claiming only a half-bath (typically a toilet/sink with no bathtub or shower). For now we'll keep these listings unchanged.

In [750]:
# Removing confirmed "no bathroom" property records.
df_NYC.drop(df_NYC[df_NYC['id'] == '21164481'].index, inplace = True)
df_NYC.drop(df_NYC[df_NYC['id'] == '3914363'].index, inplace = True)

In [751]:
# Replacing null values and remaining 0's with 1.
df_NYC['bathrooms'].fillna(1, inplace = True)
df_NYC.loc[df_NYC["bathrooms"] == 0.0,'bathrooms'] = 1

In [752]:
# Checking that zeroes and nulls have all been removed or replaced.
print(df_NYC['bathrooms'].value_counts())
print(df_NYC.bathrooms.isna().sum())

1.0    14048
2.0     1296
1.5      468
2.5      213
3.0      107
4.0       40
3.5       35
4.5       10
5.0       10
0.5        8
5.5        4
6.0        3
8.0        1
Name: bathrooms, dtype: int64
0


In [753]:
df_NYC.first_review.isna().sum()

3021

'First review' is the next column on the list of those containing nulls. It contains many, as it turns out (over 3000), which creates an interesting challenge regarding how to deal with them. 

This column contains potentially useful information, as it can indicate how long a property has been operating as a rental, 
as opposed to 'host_since', which only indicates how long a host has been hosting *any* properties on the service and not necessarily that particular property. As such, dropping the column because of the large proportion of nulls is not a good option.

Populating the nulls with a measure of the existing data (such as mean or median) is not advisable either, as we have to presume that nulls in this field are associated with a 0 in the 'number_of_reviews' column (though we'll be checking this as well). A past date for 'first_review' is incompatible with having no reviews (yet).

Only one real option remains, which is to establish a "current date", and populate all 'first_review' nulls with this date such that any diff calculation will return a zero.

To establish the "present" date, we can simply take the max of 'last_review' and add 1 day to it. Before continuing with this, however, we need to convert this column and its two companion columns ('first_review', 'last_review', and 'host_since') to pandas Datetime format so that "days since" calculations are possible.

In [754]:
df_NYC['first_review'] = pd.to_datetime(df['first_review'], format='%m/%d/%Y')
df_NYC['last_review'] = pd.to_datetime(df['last_review'], format='%m/%d/%Y')
df_NYC['host_since'] = pd.to_datetime(df['host_since'], format='%m/%d/%Y')
df_NYC.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,Luggage,Smoking,Keypad,Patio_Balcony,Garden_backyard,Private_LR,Pool,Smart lock,BBQ grill,Crib
0,6901257.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",3.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
1,6304928.0,5.129899,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",7.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
2,7919400.0,4.976734,Apartment,Entire home/apt,"TV,""Cable TV"",""Wireless Internet"",""Air conditi...",5.0,1.0,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,0,0
18,17589436.0,4.882802,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
19,18224863.0,4.595120,House,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",8.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0


In [755]:
df_NYC.loc[df_NYC['first_review'].isnull(), ['number_of_reviews']].sum()

number_of_reviews    229.0
dtype: float64

In [756]:
today = df_NYC['last_review'].max()
today += timedelta(days=1)
today

Timestamp('2017-10-03 00:00:00')

So "today" for our analysis is now established as Oct 03, 2017. This will be the date we use to populate our null values and also to anchor the calculations of our time differentials.

In [757]:
# Populating first_review nulls with today's date
df_NYC.loc[df_NYC["first_review"].isnull(),'first_review'] = today


This column of dates is not the actual measure we're interested in, however. Instead, we want to calculate the difference between these dates and today. We will create a new dataframe column called 'days_since_first_review' to accomplish this.

In [758]:
# Creating a new 'days_since_first_review' column, calculating the difference between today's date and the date of first review.
df_NYC['days_since_first_review'] = today - df_NYC['first_review']
df_NYC['days_since_first_review'].describe()
df_NYC.drop(['first_review'], axis=1, inplace = True)

count                          16243
mean     527 days 10:50:48.353136480
std      551 days 05:05:32.666871632
min                  0 days 00:00:00
25%                 58 days 00:00:00
50%                402 days 00:00:00
75%                774 days 00:00:00
max               3101 days 00:00:00
Name: days_since_first_review, dtype: object

We see that the number of days since first review ranges between 1 day and 3101 days (approximately 8.5 years), with a mean of around 22 months.

In [759]:
df_NYC.host_has_profile_pic.isna().sum()

80

In [760]:
df_NYC['host_has_profile_pic'].unique()

array(['t', nan, 'f'], dtype=object)

We now move onto the "host_has_profile_pic" column. Viewing the contents of the column, we see that it's already a t/f boolean, which is perfect for our analysis!

There are only 80 nulls, and in the absence of further information, we have to assume no data = no picture. We will update these nulls to 'f', and then convert to a 0/1 format like our other dummy variable columns.

In [761]:
df_NYC['host_has_profile_pic'].fillna('f', inplace = True)
df_NYC['host_has_profile_pic'] = np.where(df_NYC['host_has_profile_pic'].str.contains('t'), 1, 0)
df_NYC['host_has_profile_pic'].value_counts()

1    16112
0      131
Name: host_has_profile_pic, dtype: int64

In [762]:
df_NYC.host_identity_verified.isna().sum()

80

In [763]:
df_NYC['host_identity_verified'].unique()

array(['t', 'f', nan], dtype=object)

The 'host_identity_verified' column is almost exactly the same in structure as the one we just updated, right down to the 80 nulls. We'll perform the same changes on this column and move on. 

In [764]:
df_NYC['host_identity_verified'].fillna('f', inplace = True)
df_NYC['host_identity_verified'] = np.where(df_NYC['host_identity_verified'].str.contains('t'), 1, 0)
df_NYC['host_identity_verified'].value_counts()

1    10674
0     5569
Name: host_identity_verified, dtype: int64

In [765]:
df_NYC.host_response_rate.isna().sum()

4992

In [766]:
df_NYC['host_response_rate'].unique()

array([nan, '100%', '71%', '89%', '0%', '50%', '25%', '70%', '80%', '29%',
       '67%', '33%', '81%', '63%', '88%', '60%', '93%', '94%', '40%',
       '90%', '92%', '30%', '83%', '86%', '99%', '75%', '20%', '78%',
       '97%', '98%', '82%', '59%', '64%', '79%', '91%', '56%', '44%',
       '74%', '95%', '6%', '96%', '85%', '43%', '57%', '76%', '17%',
       '77%', '55%', '38%', '10%', '58%', '84%', '87%', '22%', '69%',
       '73%', '46%', '68%', '61%', '54%', '36%', '65%', '62%', '21%',
       '47%'], dtype=object)

The next column, host_response_rate has a large percentage of nulls (~5k), too many to remove. We'll instead replace these with some value.

It's not reasonable to assume that just because a host's response rate is unknown that it's actually 0% (or 100%), so a measure of existing data like mean or median is a better fit. Unfortunately, the data was formatted with a trailing percent sign, turning this numerical measure into a string instead. We'll need to strip out that character and change the column data type before proceeding with the calculation.

In [767]:
# Splitting the string at '%' and discarding everything after the separator. 
df_NYC['host_response_rate'] = df_NYC['host_response_rate'].str.split('%').str[0]
# Temporarily assigning value of -1 to nulls to allow us to change the column type to float.
df_NYC['host_response_rate'].fillna('-1.0', inplace = True)
df_NYC['host_response_rate'] = df_NYC['host_response_rate'].astype('float')
df_NYC['host_response_rate'].unique()

array([ -1., 100.,  71.,  89.,   0.,  50.,  25.,  70.,  80.,  29.,  67.,
        33.,  81.,  63.,  88.,  60.,  93.,  94.,  40.,  90.,  92.,  30.,
        83.,  86.,  99.,  75.,  20.,  78.,  97.,  98.,  82.,  59.,  64.,
        79.,  91.,  56.,  44.,  74.,  95.,   6.,  96.,  85.,  43.,  57.,
        76.,  17.,  77.,  55.,  38.,  10.,  58.,  84.,  87.,  22.,  69.,
        73.,  46.,  68.,  61.,  54.,  36.,  65.,  62.,  21.,  47.])

In [768]:
# Subsetting the data to only include "real" values, and then calculating mean and median
resp = df_NYC.loc[df_NYC["host_response_rate"] >= 0.0]
avg_resp = resp['host_response_rate'].mean()
med_resp = resp['host_response_rate'].median()
print(avg_resp)
print(med_resp)

93.76055461736735
100.0


In [769]:
# Replacing -1s (formerly nulls) with the new calculated average response rate 
df_NYC.loc[df_NYC["host_response_rate"] == -1,'host_response_rate'] = avg_resp
df_NYC['host_response_rate'].describe()

count    16243.000000
mean        93.760555
std         13.973238
min          0.000000
25%         93.760555
50%        100.000000
75%        100.000000
max        100.000000
Name: host_response_rate, dtype: float64

We've reached the second of our three date columns. 'host_since' contains valuable information, so we'll populate nulls with today's date and calculate a new difference column just like we did with 'first_review'.

In [770]:
# Populating first_review nulls with today's date
df_NYC.loc[df_NYC["host_since"].isnull(),'host_since'] = today
# Creating a new 'days_since_host_start' column, calculating the difference between today's date and the date of first review.
df_NYC['days_since_host_start'] = today - df_NYC['host_since']
df_NYC['days_since_host_start'].head()

0    2017 days
1     106 days
2     343 days
18    778 days
19    147 days
Name: days_since_host_start, dtype: timedelta64[ns]

In [771]:
df_NYC['days_since_host_start'].describe()
df_NYC.drop(['host_since'], axis=1, inplace = True)

count                           16243
mean     1230 days 23:41:44.241826800
std       696 days 21:43:56.382091912
min                   0 days 00:00:00
25%                 682 days 00:00:00
50%                1193 days 00:00:00
75%                1764 days 00:00:00
max                3500 days 00:00:00
Name: days_since_host_start, dtype: object

'last_review' is our final date column. But unlike 'first_review' and 'host_since', 'last review' doesn't provide much tangible information other than how recently the property was reviewed (with no hint as to whether this was a positive or negative review).

We were able to leverage it for our 'today' date, but there is nothing that leads us to believe this column will be a useful predictor with its current lack of context, so we'll drop it entirely.

In [772]:
# Removing the 'last_review'
df_NYC.drop(['last_review'], axis=1, inplace = True)

In [773]:
print('There are ' + str(df_NYC.neighbourhood.isna().sum()) + ' null values')

There are 2 null values


'Neighbourhood' is our next column to address. Thankfully, there are only 2 nulls in this column, so let's see if one of our other location columns can provide insight. We'll start with zipcode.

In [774]:
df_NYC.loc[df_NYC['neighbourhood'].isnull(), ['zipcode']]

,zipcode
54444,7302
64489,11363


One of the two nulls is in a zipcode that looks unfamiliar (it's presumably *0*7302 with a lost leading zero, not 7302). We can use geocoding to determine where this zip code is.

In [775]:
geolocator = Nominatim(user_agent='application')
tgt_zip = '07302'
location = geolocator.geocode(tgt_zip)
print(location.address)

Jersey City, Hudson County, New Jersey, 07302, United States


The suspect zipcode is in New Jersey, not New York! We can safely remove it. But what about the other zip?

In [776]:
df_NYC.loc[df_NYC['zipcode'] == '11363', ['neighbourhood']]

,neighbourhood
64489,NaN


In [777]:
tgt_zip = '11363'
location = geolocator.geocode(tgt_zip)
print(location.address)

Συνοικία Νέας Κυψέλης, 6η Κοινότητα Αθηνών, Δήμος Αθηναίων, Περιφερειακή Ενότητα Κεντρικού Τομέα Αθηνών, Περιφέρεια Αττικής, Αποκεντρωμένη Διοίκηση Αττικής, 11363, Ελλάς


Hmm. No other examples of that zip code in the dataset, and geocoding returns something in what looks like cyrillic. We'll have to try another tack.

We know that successive zip codes tend to be adjacent to each other, so we'll inspect the zip codes clustered around this one to see if that can help fill the gap.

In [778]:
zips = ['11361', '11362', '11363', '11364', '11365']
df_NYC.loc[df_NYC['zipcode'].isin(zips), ['zipcode', 'neighbourhood']]

,zipcode,neighbourhood
1843,11361,Bayside
4194,11361,Bayside
12315,11361,Bayside
23586,11361,Bayside
25556,11365,Flushing
26453,11361,Bayside
32228,11365,Flushing
37345,11361,Bayside
38790,11362,Queens
43316,11361,Bayside


Of the surrounding zipcodes, 2 are Bayside and 1 is Queens. We'll go with the majority and update our null to Bayside.

In [779]:
# Deleting the property located in New Jersey
df_NYC.drop(df_NYC[df_NYC['zipcode'] == '7302'].index, inplace = True)

# Replacing our remaining null with 'Bayside'
df_NYC['neighbourhood'].fillna('Bayside', inplace = True)

The 'review_scores_rating' contains the average of all the ratings associated with that property. Much like with 'first_review', we assume that nulls are related to records where 'number_of_reviews' is equal to zero (though again, we will check). And also like host_response_rate, we can't assume that just because a property hasn't been reviewed yet that we can usefully fill it with 0 or 100. 

Therefore we'll employ the same method we did with response rate and use the average.

In [780]:
df_NYC.loc[df_NYC['review_scores_rating'].isnull()]

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,Keypad,Patio_Balcony,Garden_backyard,Private_LR,Pool,Smart lock,BBQ grill,Crib,days_since_first_review,days_since_host_start
45,15238989.0,5.521461,Apartment,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",2.0,1.0,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0 days,874 days
51,17621139.0,4.753590,Apartment,Entire home/apt,"TV,Internet,""Wireless Internet"",""Air condition...",2.0,1.0,Real Bed,flexible,True,...,0,0,0,0,0,0,0,0,0 days,741 days
64,5806687.0,4.700480,Apartment,Entire home/apt,,1.0,1.0,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0 days,1276 days
66,15187281.0,5.517453,Apartment,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",5.0,1.0,Real Bed,flexible,True,...,0,0,0,0,0,0,0,0,0 days,11 days
71,14668858.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",2.0,1.0,Real Bed,moderate,False,...,0,0,0,0,0,0,0,0,0 days,622 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74011,9737352.0,4.605170,Apartment,Entire home/apt,"TV,""Wireless Internet"",Kitchen,""Free parking o...",1.0,1.0,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0 days,267 days
74071,64574.0,5.241747,Apartment,Entire home/apt,"TV,Internet,""Wireless Internet"",""Air condition...",4.0,1.0,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0 days,661 days
74080,14851530.0,4.605170,Loft,Entire home/apt,"TV,Internet,""Wireless Internet"",""Air condition...",2.0,1.0,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0 days,1044 days
74095,2278677.0,4.382027,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,1.0,Real Bed,flexible,False,...,0,0,0,0,0,0,0,0,0 days,1259 days


In [781]:
# Checking whether all nulls are associated with an as-yet-unrated property
df_NYC.loc[df_NYC['review_scores_rating'].isnull(), ['number_of_reviews']].sum()

number_of_reviews    486.0
dtype: float64

In [782]:
# Creating a subset with no null scores for the mean calculation
score = df_NYC.loc[df_NYC["review_scores_rating"].notnull()]
score

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,Keypad,Patio_Balcony,Garden_backyard,Private_LR,Pool,Smart lock,BBQ grill,Crib,days_since_first_review,days_since_host_start
0,6901257.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",3.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,472 days,2017 days
1,6304928.0,5.129899,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",7.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,59 days,106 days
2,7919400.0,4.976734,Apartment,Entire home/apt,"TV,""Cable TV"",""Wireless Internet"",""Air conditi...",5.0,1.0,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,156 days,343 days
18,17589436.0,4.882802,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,738 days,778 days
19,18224863.0,4.595120,House,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",8.0,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,65 days,147 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74066,14616148.0,4.753590,Apartment,Entire home/apt,"TV,Internet,""Wireless Internet"",""Air condition...",2.0,1.0,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,573 days,1981 days
74070,9104876.0,4.990433,Apartment,Entire home/apt,"TV,Internet,""Wireless Internet"",""Air condition...",3.0,1.0,Real Bed,flexible,True,...,0,0,0,0,0,0,0,0,151 days,973 days
74084,10387373.0,5.843544,Other,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",4.0,1.0,Real Bed,strict,False,...,0,0,0,0,0,0,0,0,282 days,1334 days
74091,3779249.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",2.0,1.0,Real Bed,moderate,False,...,0,0,0,0,0,0,0,0,108 days,1076 days


In [783]:
# Computing the mean and median scores
avg_score = score['review_scores_rating'].mean()
med_score = score['review_scores_rating'].median()
print('The mean score is ' + str(avg_score))
print('The median score is ' + str(med_score))

The mean score is 93.82690830901551
The median score is 96.0


In [784]:
df_NYC.loc[df_NYC["review_scores_rating"].isnull(),'review_scores_rating'] = avg_score
df_NYC['review_scores_rating'].describe()

count    16242.000000
mean        93.826908
std          6.702652
min         20.000000
25%         93.000000
50%         94.000000
75%         99.000000
max        100.000000
Name: review_scores_rating, dtype: float64

The next column contains url text for the thumbnail image associated with the property. We have no interest in the actual url therein, just whether the property has one or not. 

We'll do like we did with 'host_has_profile_pic' and just turn this into an 0/1 dummy variable called 'has_url' and drop the original.

In [785]:
df_NYC['has_url'] = np.where(df_NYC['thumbnail_url'].str.contains("h"), 1, 0)
df_NYC['has_url'].head()

0     1
1     1
2     1
18    1
19    1
Name: has_url, dtype: int32

In [786]:
df_NYC.drop(['thumbnail_url'], axis=1, inplace = True)

In [787]:
df_NYC['zipcode'].describe()

count     16004
unique      183
top       11211
freq        783
Name: zipcode, dtype: object

In [788]:
df_NYC['zipcode'].unique()

array(['11201', '10019', '10027', '10002', '11226', '11411', '11374',
       '10003', '10021', '10001', '10037', '10011', '10022', '10031',
       '10014', '11237', '10029', '11225', '11211', nan, '10025', '10040',
       '10038', '11238', '11206', '10032', '10012', '10023', '11221',
       '11249', '11217', '11232', '10010', '11377', '11216', '10128',
       '11207', '11229', '10314', '10009', '10282', '11218', '10036',
       '11230', '10065', '11215', '10024', '10035', '11205', '11209',
       '10030', '11213', '10016', '10475', '11101', '11231', '11103',
       '10017', '11222', '10455', '11428', '11370', '10028', '11233',
       '11204', '11372', '10039', '11417', '10004', '11102', '10018',
       '10304', '11373', '11106', '11105', '11104', '10033', '10005',
       '11433', '11355', '10013', '11223', '11367', '11379', '10463',
       '10306', '11416', '11228', '10034', '11208', '11235', '11361',
       '11385', '11354', '11220', '11419', '10026', '99135', '10457',
       '10075',

We've now reached the zipcode column we leveraged earlier for the 'neighbourhood' column. 

We immediately notice an issue in addition to the 237 null values, which is that there are a couple of entries that have the '+4' zipcode format where they have a hyphen and 4 additional numbers after the 5 target digits. We'll need to strip off these trailing characters.

Given our intention to use reverse geocoding to fill the large number of null values in this measure, we'll go ahead and create a new temporary dataframe to perform our transformation and then join this back to the original dataframe once complete.

In [789]:
df_NYC['zipcode'] = df_NYC['zipcode'].str.strip('.0')
# Creating a temporary 'zipnulls' dataframe
df_zipnulls = df_NYC.loc[df_NYC['zipcode'].isnull()]

In order to perform reverse geocoding, we need to feed the geocoder with a matched pair of lat/lon coordinates. We have both in this dataset, but they're in two separate columns. We'll use string operations to concatenate these two to create the matched pair.

In [790]:
# Combining 'latitude' and 'longitude' columns into a new 'coord' column 
df_zipnulls.loc[:,'coord'] = df_zipnulls['latitude'].astype(str) + ", " + df_zipnulls['longitude'].astype(str)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [791]:
geolocator = Nominatim(user_agent='application')
df_zipnulls['new_zip'] = df_zipnulls["coord"].apply(lambda el: geolocator.reverse(el).raw["address"]["postcode"])
print(df_zipnulls['new_zip'].value_counts())
print('There are ' + str(df_zipnulls['new_zip'].isna().sum()) + ' null values.')

10002         13
11211         12
10003         10
10036          9
10019          9
              ..
10017-6927     1
10462          1
10305          1
10039          1
11219          1
Name: new_zip, Length: 81, dtype: int64
There are 0 null values.


<ipython-input-791-e2b75c2c3241>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zipnulls['new_zip'] = df_zipnulls["coord"].apply(lambda el: geolocator.reverse(el).raw["address"]["postcode"])


In [792]:
# Subsetting 'zipnulls' to only include the 'id' and geocoding-generated 'new_zip' fields
df_newzip = df_zipnulls[['id','new_zip']]
df_newzip.head()

,id,new_zip
96,4576821.0,11215
896,15969097.0,10010
1171,18670860.0,11235
1185,19695447.0,10038
1449,20946727.0,11226


In [793]:
# Merging the newzip dataframe to the df_NYC base dataframe using the 'id' column
df_NYC = pd.merge(df_NYC, df_newzip, how='left', on='id')
df_NYC.reset_index()

,index,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,...,Garden_backyard,Private_LR,Pool,Smart lock,BBQ grill,Crib,days_since_first_review,days_since_host_start,has_url,new_zip
0,0,6901257.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",3.0,1.0,Real Bed,strict,...,0,0,0,0,0,0,472 days,2017 days,1,NaN
1,1,6304928.0,5.129899,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",7.0,1.0,Real Bed,strict,...,0,0,0,0,0,0,59 days,106 days,1,NaN
2,2,7919400.0,4.976734,Apartment,Entire home/apt,"TV,""Cable TV"",""Wireless Internet"",""Air conditi...",5.0,1.0,Real Bed,moderate,...,0,0,0,0,0,0,156 days,343 days,1,NaN
3,3,17589436.0,4.882802,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,1.0,Real Bed,strict,...,0,0,0,0,0,0,738 days,778 days,1,NaN
4,4,18224863.0,4.595120,House,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",8.0,1.0,Real Bed,strict,...,0,0,0,0,0,0,65 days,147 days,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16237,16237,10387373.0,5.843544,Other,Entire home/apt,"TV,""Wireless Internet"",""Air conditioning"",Kitc...",4.0,1.0,Real Bed,strict,...,0,0,0,0,0,0,282 days,1334 days,1,NaN
16238,16238,3779249.0,5.010635,Apartment,Entire home/apt,"""Wireless Internet"",""Air conditioning"",Kitchen...",2.0,1.0,Real Bed,moderate,...,0,0,0,0,0,0,108 days,1076 days,1,NaN
16239,16239,2278677.0,4.382027,Apartment,Entire home/apt,"TV,""Cable TV"",Internet,""Wireless Internet"",""Ai...",2.0,1.0,Real Bed,flexible,...,0,0,0,0,0,0,0 days,1259 days,1,NaN
16240,16240,18688039.0,5.220356,Apartment,Entire home/apt,"TV,Internet,""Wireless Internet"",""Air condition...",5.0,1.0,Real Bed,moderate,...,0,0,0,0,0,0,1004 days,2098 days,1,NaN


In [794]:
# Replacing null values in the 'zipcode' column with their new generated zip in the 'newzip' column
df_NYC.loc[df_NYC["zipcode"].isnull(),'zipcode'] = df_NYC["new_zip"]

# Removing the '+4' characters from the zipcodes that contain them
df_NYC['zipcode'] = df_NYC['zipcode'].str.split('-').str[0]

# Dropping the 'new_zip' column
df_NYC.drop(['new_zip'], axis=1, inplace = True)

# Verifying that the null values have all been updated
print('There are ' + str(df_NYC['zipcode'].isna().sum()) + ' null values.')

There are 0 null values.


In [795]:
print('There are ' + str(df_NYC['beds'].isna().sum()) + ' null values.')

There are 10 null values.


The last column that needs to be addressed is 'beds'. There are only 10 null values, and common sense dictates that we simply assume that there is a 1:1 bedroom relationship and just 'borrow' the value from the 'bedrooms' column for those 10 rows.

In [796]:
df_NYC.loc[df_NYC["beds"].isnull(),'beds'] = df_NYC["bedrooms"]
print('There are ' + str(df_NYC['beds'].isna().sum()) + ' null values.')

There are 0 null values.


In [797]:
df_NYC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16242 entries, 0 to 16241
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   id                       16242 non-null  float64        
 1   log_price                16242 non-null  float64        
 2   property_type            16242 non-null  object         
 3   room_type                16242 non-null  object         
 4   amenities                16242 non-null  object         
 5   accommodates             16242 non-null  float64        
 6   bathrooms                16242 non-null  float64        
 7   bed_type                 16242 non-null  object         
 8   cancellation_policy      16242 non-null  object         
 9   cleaning_fee             16242 non-null  object         
 10  city                     16242 non-null  object         
 11  description              16242 non-null  object         
 12  first_review      

We've made a lot of progress with this data, and are left with something a lot more workable.

We broke out all of the possible amenities from that column, categorized them, and then created new dummy variable columns for the ones we intend to leverage.

We dealt with null values in 12 of our columns, and concurrently verified data fidelity for those columns as well.

Up next, we'll dive deeper into our data during our Exploratory Data Analysis step.